#### <p style="text-align:center;">Welcome to the <b><span style="color:green">Upply Notebook</span></b>, a notebook dedicated to using OpenAI's ChatGPT (4o-mini) to help generate text for proposals on the freelance site 'Upwork'.</p> 
##### <p style="text-align:center;">The purpose of this notebook is to <i><span style="color:red">generate seemless prompts</span></i> that are good enough to feel organic and natural, and in no-way AI generated.</p>
#### <p style="text-align:center;"><u>A breakdown of what this entails:</u></p>
#### 1.) Bringing in a Markdown version of my resume for ChatGPT to reference;
#### 2.) Taking in the Job Description from the Upwork Website Via Copy/Paste;
#### 3.) Constructing A Prompt Template with ChatGPT; and
#### 4.) Generating A Natural-Sounding Proposal
***

In [1]:
# First and foremost, though: import the python libraries we'll need

# to access environmental openai key variable housed in .env file
import os
from dotenv import load_dotenv

# check - should come back 'True'
# load_dotenv()

# for rich displays in Markdown becuase it is dynamic, unboring text
from IPython.display import display, Markdown

# use Python's OpenAI package to access the OpenAI API
from openai import OpenAI

# we want to convert the markdown to HTML for the site's use
from markdown import markdown

***
#### Step 1: Bringing in a Markdown version of my resume for ChatGPT to reference
***

In [2]:
# open up the saved Markdown resume and read it
with open("/Users/nicholasjoseph/Desktop/nj_pm_res.md", "r", encoding = "utf-8") as resume_file:
    resume_string = resume_file.read()

# check
# display(Markdown(resume_string))

##### the check looked good. python's reading the resume as expected. Now onto:
***
#### Step 2: Taking In the Job Description from the Upwork Website Via Copy / Paste
***

In [3]:
# simple enough because it'll be a physical copy / paste
job_description_string = input()

 Needs to hire 4 Freelancers We create free, advanced educational content for venture-backed technology startups in the public cloud and IT infrastructure space. The readers of our content include administrators, engineers, and technical managers learning to master key industry technologies.  To produce content superior to the usual online material, we rely on subject matter experts like you who are willing to share their knowledge with less experienced peers to lend a genuine and practical voice to our educational articles.  Our primary interest is your knowledge, ideas, technical expertise, and your ability to communicate them to readers in the way a mentor might train a new team member. We work with professional editors and graphic designers, so we don’t need perfect English or drawing skills, but we require deep subject matter expertise and the ability to articulate your ideas clearly in grammatically correct sentences.  The articles below are examples of the type of content we cre

***
#### Step 3: Constructing A Prompt Template with ChatGPT
##### 'lambda' function is being used here because making the return statement all the prompt text seems ridiculous.
##### also, this is a template I prompted ChatGPT with - reason being, let Chat generate it because LLMs talk to LLMs. 
***

In [4]:
# essentially, this is a large formatted string assigned to a single variable
prompt_template = lambda resume_string, job_description_string : f"""
You are a professional proposal writer writing for clients looking for freelance work on Upwork.

Your job is to read a freelancer's resume and a specific job description, then craft a personalized proposal for that job. The proposal should highlight the freelancer's most relevant experience, skills, and accomplishments as they relate to the job description.

Use the following guidelines:

1. **Tone**: Blend personable warmth with technical competence. Show that the freelancer is both approachable and highly capable.
2. **Content focus**: Prioritize skills, tools, and accomplishments that most closely match the job description.
3. **Language**: Use confident, active, and natural language with strong action verbs and quantifiable results wherever possible (e.g., "boosted revenue by 40%", "reduced processing time by 60%", etc.).
4. **Length**: Keep the proposal concise (under 200 words), but meaningful.
5. **Close strong**: End with a short paragraph inviting the client to connect, ask questions, or discuss the project further.
6. **Integrate **keywords** and phrases from the job description naturally to give the text an 'I-read-over-your-post' type of feel.
7. **Output the tailored resume in the appropriate Upwork proposal format that is clean and free from errors.

---

INPUTS:

RESUME:
<<<
{resume_string}
>>>

JOB DESCRIPTION:
<<<
{job_description_string}
>>>

---

TASK:
Write a complete Upwork proposal tailored to the job description above. Emphasize alignment between the freelancer's background and the project requirements. Don't include unnecessary fluff—make every sentence purposeful and impactful.

"""

In [5]:
prompt = prompt_template(resume_string, job_description_string)

***
#### Step 4: Generating A Natural-Sounding Proposal
***

In [6]:
# set up the api clent

open_apikey = os.environ.get("openai_apikey")

client = OpenAI(api_key = open_apikey)

# call the api
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role" : "system", "content" : "Expert Proposal Writer"},
        {"role" : "user", "content" : prompt}
    ], 
    temperature = 0.7
)

response
# see the response
# response_string = response.choices[0].message.content

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-Zc1TwXtDpPr0IS6RN5ix57gZ on tokens per min (TPM): Limit 100000, Used 99610, Requested 1994. Please try again in 11h32m55.68s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
type(response)

***
#### Since the response is "openai.types.chat.chat_completion.ChatCompletion" and we want to put this into a markdown file
#### for editing, we want to extract the relevant data, iterate through choices, format said choices, and finally create
#### the Markdown file for export.
***

In [ ]:
# create the response_data dictionary to hold the info
response_data = {
    "id": response.id,
    "model": response.model,
    "created": response.created,
    "object": response.object,
    "choices": []
}

# loop for going through the choices and formatting them
for choice in response.choices:
    choice_data = {
        "index": choice.index,
        "finish_reason": choice.finish_reason,
        "role": choice.message.role,
        "content": choice.message.content
    }
    response_data["choices"].append(choice_data)

# create the backbone of the markdown with the content in a codeblock for readability
markdown_content = "# OpenAI Chat Completion Response\n\n"

markdown_content += f"- **ID:** {response_data['id']}\n"
markdown_content += f"- **Model:** {response_data['model']}\n"
markdown_content += f"- **Created (Timestamp):** {response_data['created']}\n"
markdown_content += f"- **Object:** {response_data['object']}\n\n"

markdown_content += "## Choices:\n\n"
for choice in response_data['choices']:
    markdown_content += f"### Choice {choice['index'] + 1}\n\n"
    markdown_content += f"- **Finish Reason:** {choice['finish_reason']}\n"
    markdown_content += f"- **Role:** {choice['role']}\n"
    markdown_content += f"- **Content:**\n"
    markdown_content += "```\n"
    markdown_content += choice['content'] + "\n"
    markdown_content += "```\n\n"

# create the location
output_filename = "upwork_proposal.md"
output_path = os.path.join(os.getcwd(), output_filename)

# write the file
with open(output_path, "w") as f:
    f.write(markdown_content)

print(f"OpenAI Chat Completion response exported to: {output_path}")